In [1]:
#GP

#starting with large set of programs (population)
#programs 'compete' in a task
#ranked list of performance obtained
#best programs are replicated and modified
#either via mutation (random change)
#or by crossover (swapping parts with other 'good programs')

#quality measured by a 'fitness function'

#need a way to represent programs with Python code
#most programming languages are turned into a parse tree

#each node in the tree represents an operation on it's children
#or is a root which represents a value

class FWrapper:
    def __init__(self,function,child_count,name):
        self.function=function
        self.child_count=child_count
        self.name=name

class Node:
    def __init__(self,fw,children,parent=None):
        self.function=fw.function
        self.name=fw.name
        self.children=children
        self.parent=parent

    def evaluate(self,inp):    
        results=[n.evaluate(inp) for n in self.children]
        return self.function(results)
    
    def display(self,indent=0):
        print((' '*indent)+self.name)
    
        for c in self.children:
            c.display(indent+1)
    

class ParamNode:
    def __init__(self,idx):
        self.idx=idx

    def evaluate(self,inp):
        return inp[self.idx]
    def display(self,indent=0):
        print('%sp%d' % (' '*indent,self.idx))
    
    
class ConstNode:
    def __init__(self,v):
        self.v=v
    def evaluate(self,inp):
        return self.v
    def display(self,indent=0):
        print('%s%d' % (' '*indent,self.v))

In [2]:
#also need some functions for the nodes to apply.
#create functions, give them names and parameters counts

add_w=FWrapper(lambda l:l[0]+l[1],2,'add')
sub_w=FWrapper(lambda l:l[0]-l[1],2,'sub') 
mul_w=FWrapper(lambda l:l[0]*l[1],2,'mul')

def if_func(l):
    if l[0]>0: return l[1]
    else: return l[2]

if_w = FWrapper(if_func,3,'if')

def gt_func(l):
    if l[0]>l[1]: return 1
    else: return 0
gt_w=FWrapper(gt_func,2,'gt')

f_list=[add_w,mul_w,if_w,gt_w,sub_w]

In [3]:
def example_tree():
    return Node(if_w,[
                    Node(gt_w,[ParamNode(0),ConstNode(3)]),
                    Node(add_w,[ParamNode(1),ConstNode(5)]),
                    Node(sub_w,[ParamNode(1),ConstNode(2)]),
                    ]
               )

#f(x,y) -> if x > 3: 
#            return y+5
#          else:
#            return 1-2

In [4]:
example_tree = example_tree()

In [5]:
example_tree.evaluate([5,3]) #ParamNode(0) = 5, ParamNode(1) = 3

8

In [6]:
example_tree.display()

if
 gt
  p0
  3
 add
  p1
  5
 sub
  p1
  2


In [7]:
#we initialise the population randomly as this introduces more diversity
#into the initial population

#creating random programs consists of creating a root node with a random
#function and then creating as many random child nodes as necessary, 
#these children may also have their own random child nodes

#like most functions with trees, this is also easily defined recursively

import random

#pc is parameter count
#fpr is probability of generating a function
#ppr is probability of generating a parameter (if not a function)

def make_random_tree(pc,max_depth=4,fpr=0.5,ppr=0.6):
    if random.random()<fpr and max_depth>0:
        f=random.choice(f_list)
        children=[make_random_tree(pc,max_depth-1,fpr,ppr) 
              for i in range(f.child_count)]
        return Node(f,children)
    elif random.random()<ppr:
        return ParamNode(random.randint(0,pc-1))
    else:
        return ConstNode(random.randint(0,10))
    
#creates a node with a random function, then looks to see how many child
#nodes this function requires
#for every child node required, the function calls itself to create a new
#node
#entire tree constructed when the function requires no more child nodes

In [8]:
program1=make_random_tree(2)

In [9]:
program1.display()

p1


In [10]:
program1.evaluate([52,50])

50

In [11]:
#some trees get quite deep since each branch will keep growing until it hits
#a zero-child node (leaf/terminal), this is why you include a max_depth

In [12]:
#now, to use GP, get given some examples from X to Y

#this is the function you want to work out

def hidden_function(x,y):
    return x**2+2*y+3*x+5

In [13]:
#use this function to build a dataset which you can test your generated programs

def build_hidden_set(target_function):
    rows = []
    for i in range(200):
        x = random.randint(0, 40)
        y = random.randint(0, 40)
        rows.append([x, y, target_function(x,y)])
    return rows

In [14]:
hidden_set = build_hidden_set(hidden_function)

print(hidden_set[0])

[8, 21, 135]


In [15]:
#as we are testing a program with a numerical outcome
#the easiest way to test a program is to see how close it gets to the 
#correct answers

def score_function(tree,s):
    loss=0
    for data in s:
        v=tree.evaluate([data[0],data[1]])
        loss+=abs(v-data[2])
    return loss

In [16]:
score_function(program1, hidden_set)

127176

In [17]:
#we now have a way to determine how good our function is
#this is important for deciding which programs make it to the next generation

#mutation takes a single program and alters it slightly
#either by changing a node or altering branches

#shouldn't mutate too much as randomly changing things has a pretty low
#chance of actually improving the outcome
#instead assign a small probability a mutation will happen

#the code below performs sub-tree mutation

import copy

def mutate(t,pc,prob=0.1):
    if random.random()<prob:
        return make_random_tree(pc)
    else:
        result=copy.deepcopy(t)
        if hasattr(t,"children"):
            result.children=[mutate(c,pc,prob) for c in t.children]
        return result
    
#the function begins at the top of the tree and decides if a node should 
#be altered, if not it calls mutate on the child nodes of a tree

In [18]:
program1.display()

p1


In [19]:
mutated_program_1 = mutate(program1, 2)
mutated_program_1.display()

p1


In [20]:
mutated_program_1.evaluate([-10,5])

5

In [21]:
#the other type of program modification is crossover
#this takes two successful programs and combines to create a new program
#by replacing a branch from one with a branch from another

#crossover takes two trees and inputs and traverses both of them
#when a threshold is reached the function returns a copy of the first tree
#with one branch replaced by a branch in the second tree

#by traversing both trees at once, crossover happens at approximately the
#same level on each tree

def crossover(t1,t2,prob=0.7,top=1):
    if random.random()<prob and not top:
        return copy.deepcopy(t2) 
    else:
        result=copy.deepcopy(t1)
    if hasattr(t1,'children') and hasattr(t2,'children'):
        result.children=[crossover(c,random.choice(t2.children),prob,0) 
                       for c in t1.children]
    return result

In [22]:
prog1 = make_random_tree(2)
prog1.display()

sub
 sub
  3
  add
   5
   7
 gt
  p0
  sub
   5
   7


In [23]:
prog2 = make_random_tree(2)
prog2.display()

sub
 if
  if
   if
    p1
    3
    10
   add
    p1
    p0
   p0
  sub
   if
    7
    6
    0
   mul
    p1
    p0
  6
 p0


In [24]:
prog3 = crossover(prog2, prog1)
prog3.display()

sub
 sub
  3
  add
   5
   7
 sub
  3
  add
   5
   7


In [25]:
#need to rank programs based on the result of score_function
#get_rank_function returns a ranking function for a given dataset

def get_rank_function(dataset):
    def rank_function(population):
        scores=[(score_function(t,dataset),t) for t in population]
        scores = sorted(scores, key= lambda x: x[0])
        return scores
    return rank_function

In [ ]:
"""#now ready to set up an environment to evolve best programs
#create a set of random programs
#select the best ones for replication and modification
#repeat until some stopping criteria is reached

import math

def evolve(pc, pop_size, rank_function, max_gen=500, p_mutation=0.1,
           p_crossover=0.4, exp=0.7, p_new=0.05):
    #returns a random number tending towards lower numbers
    #the lower exp is, the more lower numbers you get
    def select_index():
        return int(math.log(random.random())/math.log(exp))
    
    #create random initial population
    pop=[make_random_tree(pc) for i in range(pop_size)]
    for i in range(max_gen):
        scores = rank_function(pop)
        print("best score: ",scores[0][0])
        if scores[0][0] == 0:
            break
            
        #two best always make it
        new_pop = [scores[0][1], scores[1][1]]
        
        #build next generation
        while len(new_pop)<pop_size:
            if random.random()>p_new:
                new_pop.append(mutate(crossover(scores[select_index()][1],
                                                scores[select_index()][1],
                                                prob=p_crossover),
                                      pc,
                                      prob=p_mutation))
            else: 
                #add a random tree to shake things up
                new_pop.append(make_random_tree(pc))
                
        population = new_pop
    scores[0][1].display()
    return scores[0][1]

#creates an initial random population
#loops up to max_gen times
#each time calling rank_function to rank programs best to worst
#the best program is automatically passed through to the next generation
#unaltered (this is called 'elitism')
#rest of the population is constructed by randomly choosing programs near
#the top of the ranking and then breeding and mutating them
#this repeats until perfect score of 0 or max_gen is reached

"""
#now ready to set up an environment to evolve best programs
#create a set of random programs
#select the best ones for replication and modification
#repeat until some stopping criteria is reached

import math

def evolve(pc,popsize,rankfunction,max_gen=500,
           p_mutation=0.1,p_crossover=0.4,exp=0.7,p_new=0.05):
    
    count = 0
    
  # Returns a random number, tending towards lower numbers. The lower pexp
  # is, more lower numbers you will get
    def selectindex():
        return int(math.log(random.random())/math.log(exp))

  # Create a random initial population
    population=[make_random_tree(pc) for i in range(popsize)]
    for i in range(max_gen):
        scores=rankfunction(population)
        print(scores[0][0])
        count += 1
        if scores[0][0]==0: break
    
        # The two best always make it
        newpop=[scores[0][1],scores[1][1]]
    
        # Build the next generation
        while len(newpop)<popsize:
            if random.random()>p_new:
                newpop.append(mutate(
                      crossover(scores[selectindex()][1],
                                 scores[selectindex()][1],
                                prob=p_crossover),
                        pc,prob=p_mutation))
            else:
            # Add a random node to mix things up
                newpop.append(make_random_tree(pc))
        
        population=newpop
    #scores[0][1].display()    
    return scores[0][1], count

#creates an initial random population
#loops up to max_gen times
#each time calling rank_function to rank programs best to worst
#the best program is automatically passed through to the next generation
#unaltered (this is called 'elitism')
#rest of the population is constructed by randomly choosing programs near
#the top of the ranking and then breeding and mutating them
#this repeats until perfect score of 0 or max_gen is reached

In [ ]:
rf = get_rank_function(build_hidden_set(hidden_function))

total = 0

for i in range(10):
    _, count = evolve(2, 500, rf, max_gen=10, p_mutation=0.1, p_crossover=0.5, exp=0.7, p_new=0.01)
    total += count
    print("count:",count)
print("avg count: ",total/10)

12546
6768
4710
2859
2102
2023
2023
2023
2022
2022
count: 10
20343
10420
5260
4885
4308
4308
4260
4260
4260
4260
count: 10
20343
12845
5504
5504
5504
5462
5369
4082
2731
991
count: 10
14183
5204
3910
490
225
225
200
193
193
193
count: 10
20343
5504
4420
4420
4420
4420
3869


In [ ]:
import gp

In [ ]:
rf=gp.getrankfunction(gp.buildhiddenset())
gp.evolve(2, 500, rf, mutationrate=0.1, breedingrate=0.5, pexp=0.7, pnew=0.01)

total = 0

for i in range(10):
    _, count = gp.evolve(2, 500, rf, maxgen=10, mutationrate=0.1, breedingrate=0.5, pexp=0.7, pnew=0.01)
    total += count
    print("count:",count)
print("avg gens: ",total/10)